<div align = "center">

# **Limpieza extra para Frecuencia-Severidad**

</div>

## Librerias

In [14]:
library(dplyr)
library(tidyr)

## Data

In [15]:
datos_limpios <- read.csv("../data/processed/datos_limpios.csv")

## Agrupando

In [16]:
# Variables categóricas relevantes para el modelo
variables_tarifa <- c("Modelo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                      "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg")

# Crear variables de edad categóricas basadas en las dummies
datos_limpios <- datos_limpios %>%
  mutate(
    Grupo_Edad = case_when(
      Edad_19_41 == 1 ~ "19-41",
      Edad_41_63 == 1 ~ "41-63", 
      Edad_63_85 == 1 ~ "63-85",
      TRUE ~ "Otro"  # Por si hay casos fuera de estos rangos
    )
  )

# Agregar datos por celda tarifaria
tabla_agregada <- datos_limpios %>%
  group_by(across(all_of(c(variables_tarifa, "Grupo_Edad")))) %>%
  summarize(
    # Para el modelo de frecuencia
    n_siniestros = sum(Accidentado),
    exposicion_total = sum(exposicion),
    
    # Para el modelo de severidad
    suma_pagos = sum(SumaDePagos),
    n_observaciones = n(),
    
    .groups = 'drop'
  )

# Convertir variables categóricas a factores en tabla_agregada
tabla_agregada <- tabla_agregada %>%
  mutate(
    Color = as.factor(Color),
    Carroceria = as.factor(Carroceria),
    CLASE_FASECOLDA = as.factor(CLASE_FASECOLDA),
    TIPO_VEHICULO = as.factor(TIPO_VEHICULO),
    SERVICIO = as.factor(SERVICIO),
    Sexo_Aseg = as.factor(Sexo_Aseg),
    Grupo_Edad = as.factor(Grupo_Edad)
  )

# Crear subset para severidad (solo celdas con siniestros)
tabla_severidad <- tabla_agregada %>%
  filter(n_siniestros > 0) %>%
  mutate(
    severidad_media = suma_pagos / n_siniestros,
    # Mantener todos los niveles de los factores aunque no aparezcan en los datos filtrados
    across(where(is.factor), ~factor(., levels = levels(.)))
  )

In [17]:
head(tabla_agregada)

Modelo,Color,Carroceria,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Grupo_Edad,n_siniestros,exposicion_total,suma_pagos,n_observaciones
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<dbl>,<int>
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,F,41-63,2,1834,2558114,10
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19-41,2,1203,4883644,7
1995,VIOLETA,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,0,3341,0,19
1997,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,41-63,2,3077,4238816,16
1998,AZUL,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,2,2791,8213167,16
1998,AZUL,SEDAN,AUTOMOVIL,Livianos,Particular,F,41-63,0,1466,0,8


In [18]:
head(tabla_severidad)

Modelo,Color,Carroceria,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Grupo_Edad,n_siniestros,exposicion_total,suma_pagos,n_observaciones,severidad_media
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,F,41-63,2,1834,2558114,10,1279057
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19-41,2,1203,4883644,7,2441822
1997,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,41-63,2,3077,4238816,16,2119408
1998,AZUL,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,2,2791,8213167,16,4106584
1998,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,2,2410,8014069,13,4007035
1998,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19-41,4,4223,6450375,23,1612594


In [19]:
# Primero, veamos qué tipo de variables tienes
str(tabla_agregada)

tibble [449 × 12] (S3: tbl_df/tbl/data.frame)
 $ Modelo          : int [1:449] 1993 1993 1995 1997 1998 1998 1998 1998 1998 1998 ...
 $ Color           : Factor w/ 25 levels "AMARILLO","ARENA FINA",..: 19 19 25 5 3 3 5 5 5 11 ...
 $ Carroceria      : Factor w/ 11 levels "BUS-BUSETA","CABINADO",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ CLASE_FASECOLDA : Factor w/ 8 levels "AUTOMOVIL","AUTOMOVIL TAXI",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ TIPO_VEHICULO   : Factor w/ 2 levels "Livianos","Pesados": 1 1 1 1 1 1 1 1 1 1 ...
 $ SERVICIO        : Factor w/ 3 levels "Particular","Publico",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Sexo_Aseg       : Factor w/ 2 levels "F","M": 1 2 1 2 1 1 1 1 2 2 ...
 $ Grupo_Edad      : Factor w/ 3 levels "19-41","41-63",..: 2 1 1 2 1 2 1 2 1 1 ...
 $ n_siniestros    : int [1:449] 2 2 0 2 2 0 2 0 4 0 ...
 $ exposicion_total: int [1:449] 1834 1203 3341 3077 2791 1466 2410 1467 4223 2934 ...
 $ suma_pagos      : num [1:449] 2558114 4883644 0 4238816 8213167 ...
 $ n_observaciones : int [1:449

In [20]:
write.csv(tabla_severidad, "../data/processed/tabla_severidad.csv")